#### Introduce package

In [21]:
from ase.io.cube import read_cube_data,read_cube

#### Data loading 

In [23]:
data = read_cube(open('qe_files/n2/output/homo-restricted.cube'))


In [32]:
print(data.keys())
print(data['atoms'])
print(data['origin'])
print(data['labels'])
print(data['datas'].shape)
print(data['data'].shape)

dict_keys(['atoms', 'data', 'origin', 'spacing', 'labels', 'datas'])
Atoms(symbols='LiH', pbc=True, cell=[15.000025460010008, 15.000025460010008, 15.000025460010008])
[-7.4999995 -7.4999995 -7.4999995]
[1]
(1, 140, 140, 140)
(140, 140, 140)


#### one-electron integrals

In [1]:
def calc_h_ij(self, p, c_ip_up, c_ip_dw, atoms, cell_volume):
    # Kinetic energy
    iTj_up = calc_matrix_elements.iTj(self.p, self.c_ip_up)
    iTj_dw = calc_matrix_elements.iTj(self.p, self.c_ip_dw)

    # Nuclear repulsion
    iUj_up = calc_matrix_elements.iUj(
        p,
        c_ip_up,
        atoms,
        cell_volume,
    )
    iUj_dw = calc_matrix_elements.iUj(
        p,
        c_ip_dw,
        atoms,
        cell_volume,
    )

    h_ij_up = iTj_up - iUj_up
    h_ij_dw = iTj_dw - iUj_dw

    return h_ij_up, h_ij_dw

#### Two-electron integrals

In [ ]:
def calc_eri(self):
    # Calculate ERIs via pair density
    assert (
        self.wfc_up_obj.gamma_only
    ), "Calculating ERIs via pair densities is only implemented for the gamma-point!"
    assert (
        self.wfc_dw_obj.gamma_only
    ), "Calculating ERIs via pair densities is only implemented for the gamma-point!"

    eri_up: np.ndarray = (
        eri_pair_densities.eri_gamma(p=self.p, c_ip_up=self.c_ip_up)
        / self.wfc_up_obj.cell_volume
    )
    eri_dw: np.ndarray = (
        eri_pair_densities.eri_gamma(p=self.p, c_ip_up=self.c_ip_dw)
        / self.wfc_dw_obj.cell_volume
    )
    eri_dw_up: np.ndarray = (
        eri_pair_densities.eri_gamma(
            p=self.p, c_ip_up=self.c_ip_up, c_ip_dw=self.c_ip_dw
        )
        / self.wfc_dw_obj.cell_volume
    )
    # eri_up_dw: np.ndarray = (
    #     eri_pair_densities.eri_gamma(
    #         p=self.p, c_ip_up=self.c_ip_dw, c_ip_dw=self.c_ip_up
    #     )
    #     / self.wfc_dw_obj.cell_volume
    # )
    eri_up_dw: np.ndarray = eri_dw_up.swapaxes(0, 1).swapaxes(2, 3)

    return eri_up, eri_dw, eri_dw_up, eri_up_dw